In [1]:
import tools
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier

In [13]:
features_train = tools.features_train('data/features_train.csv')

In [3]:
features_train

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,radiant_win
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0,1
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0,0
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0,0
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114402,1450265551,1,47,4,1706,1198,17,0,1,8,...,-29.0,180.0,-76.0,180.0,3,4,3,0,-24.0,0
114403,1450277704,0,43,4,1793,1416,17,0,1,5,...,-5.0,NaN,-82.0,NaN,4,3,2,0,-17.0,1
114404,1450291848,1,98,4,1399,540,1,0,0,5,...,-32.0,249.0,-70.0,NaN,1,1,3,1,-15.0,0


In [4]:
features_train.shape

(97230, 103)

In [5]:
features_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97230 entries, 0 to 114406
Columns: 103 entries, start_time to radiant_win
dtypes: float64(12), int64(91)
memory usage: 77.1 MB


In [14]:
counts = features_train.count()
counts[counts != y.size]

first_blood_time               77677
first_blood_team               77677
first_blood_player1            77677
first_blood_player2            53243
radiant_bottle_time            81539
radiant_courier_time           96538
radiant_flying_courier_time    69751
radiant_first_ward_time        95394
dire_bottle_time               81087
dire_courier_time              96554
dire_flying_courier_time       71132
dire_first_ward_time           95404
dtype: int64

In [16]:
for idx in counts[counts != features_train.shape[0]].index:
    features_train[idx].fillna(features_train[idx].value_counts().index[0], inplace=True)

In [17]:
x, y = features_train.iloc[:, :-1], features_train.iloc[:, -1]

In [ ]:
[0.66497345 0.66500504 0.66470694 0.662281   0.66719818]

In [20]:
cross_val_score?

Signature:
cross_val_score(
    estimator,
    X,
    y=None,
    *,
    groups=None,
    scoring=None,
    cv=None,
    n_jobs=None,
    verbose=0,
    fit_params=None,
    pre_dispatch='2*n_jobs',
    error_score=nan,
)
Docstring:
Evaluate a score by cross-validation

Read more in the :ref:`User Guide <cross_validation>`.

Parameters
----------
estimator : estimator object implementing 'fit'
    The object to use to fit the data.

X : array-like of shape (n_samples, n_features)
    The data to fit. Can be for example a list, or an array.

y : array-like of shape (n_samples,) or (n_samples, n_outputs),             default=None
    The target variable to try to predict in the case of
    supervised learning.

groups : array-like of shape (n_samples,), default=None
    Group labels for the samples used while splitting the dataset into
    train/test set. Only used in conjunction with a "Group" :term:`cv`
    instance (e.g., :class:`GroupKFold`).

scoring : str or callable, default=None


In [21]:
kf = KFold(n_splits=5, shuffle=True, random_state=1)

array([    1,     4,     6, ..., 97206, 97223, 97226])

In [37]:
cv_scores = []
kf = KFold(n_splits=5, shuffle=True, random_state=1)
for n in [10, 250]:
    clf = GradientBoostingClassifier(n_estimators=n, learning_rate=1e-3, random_state=1)
    print('#', str(n))
    
    start_time = pd.datetime.now()
    model_scores = cross_val_score(clf, x, y, cv=kf, scoring='roc_auc', n_jobs=4)
    print('Time elapsed:', pd.datetime.now() - start_time, '\n', model_scores)
    
    cv_scores.append(model_scores)

# 10
Time elapsed: 0:00:24.124999 
 [0.58081165 0.60208098 0.59429532 0.60073674 0.62023546]
# 250
Time elapsed: 0:08:41.139001 
 [0.61869441 0.62611707 0.62342265 0.62237721 0.63128484]
